# Notebook 4: Model Development, Optimisation, and Interpretation

**Notebook Purpose**
This notebook develops, optimizes, and interprets the main predictive models:
1. Train multiple model types (including Random Forest, XGBoost, Neural Networks)
2. Hyperparameter tuning using validation set performance
3. Cross-validation with TimeSeriesSplit to respect temporal ordering
4. Model comparison and selection based on validation metrics
5. Feature importance analysis to identify which spectral bands predict oxygen

**Key Outputs**
- `best_model.pkl`: Trained best-performing model
- Feature importance rankings and visualizations (SHAP, permutation importance)
- Validation set performance metrics
- Insights about satellite derived features-oxygen relationships

**Data Leakage Prevention**
All training, hyperparameter tuning, and model selection use validation set only. TimeSeriesSplit prevents look-ahead bias in cross-validation. Test set remains unsued until Notebook 5.

## Library Imports

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Ingestion

In [ ]:
training_set_file_path = '/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Data Splits/Processed/Training Set.csv'
validation_set_file_path = '/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Data Splits/Processed/Validation Set.csv'
testing_set_file_path = '/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Data Splits/Processed/Testing Set.csv'

training_df = pd.read_csv(training_set_file_path)
validation_df = pd.read_csv(validation_set_file_path)
testing_df = pd.read_csv(testing_set_file_path)

training_df.head()
# validation_df.head()
# testing_df.head()

,Latitude,Longitude,Year,Month,Day,chlor_a,poc,sst,Rrs_412,Rrs_443,...,sst_squared,sst_cubed,log_chlor_a,log_poc,ratio_443_547,ratio_443_555,sst_chlor_interaction,abs_latitude,season,Oxygen
0,54.6230,13.0280,2002,7,4,3.398178,274.399994,15.715000,0.002476,0.001970,...,246.961230,3880.995764,1.223239,5.614587,0.714286,0.755368,19.223206,54.6230,Summer,278.0
1,54.5960,18.7737,2002,7,4,3.311782,242.399994,16.369999,0.002618,0.002244,...,267.976865,4386.780994,1.197486,5.490589,0.804301,0.843609,19.602851,54.5960,Summer,322.0
2,54.5777,18.7477,2002,7,4,5.786841,294.200012,16.100000,0.002444,0.001922,...,259.210012,4173.281297,1.755587,5.684260,0.669687,0.704029,28.264945,54.5777,Summer,320.0
3,54.5700,18.6800,2002,7,4,5.830627,295.600006,16.010000,0.002370,0.001860,...,256.320107,4103.684977,1.763125,5.689007,0.665712,0.700301,28.227624,54.5700,Summer,328.0
4,54.5782,18.6610,2002,7,4,5.830627,295.600006,16.010000,0.002370,0.001860,...,256.320107,4103.684977,1.763125,5.689007,0.665712,0.700301,28.227624,54.5782,Summer,328.0


# Helper functions

In [ ]:
def add_datetime_and_sort(df):
    df = df.copy()
    df["__dt__"] = pd.to_datetime(
        dict(year=df["Year"], month=df["Month"], day=df["Day"]),
        errors='coerce'
    )
    df = df.sort_values("__dt__").reset_index(drop=True)
    return df

In [ ]:
training_df,validation_df = add_datetime_and_sort(training_df),add_datetime_and_sort(validation_df)

## Advanced Models

Linear Regression variant: elastic net We chose to switch to elastic net here as it is more tuneable

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

import numpy as np


def elastic_net_model(training_df,validation_df,target_col='Oxygen'):
  X_train=training_df.drop(target_col, axis=1)

  y_train=training_df[target_col]

  X_val=validation_df.drop(target_col, axis=1)

  y_val=validation_df[target_col]

  preprocessor = ColumnTransformer(
      transformers=[
          ("num", Pipeline([
              ("imputer", SimpleImputer(strategy="median")),
              ("scaler", StandardScaler())
          ]), selector(dtype_include=np.number)),
          ("cat", Pipeline([
              ("imputer", SimpleImputer(strategy="most_frequent")),
              ("ohe", OneHotEncoder(handle_unknown="ignore"))
          ]), selector(dtype_exclude=np.number)),
      ],
      remainder="drop"
  )



  elastic_net = ElasticNet()
  model = Pipeline([
      ("preprocessor", preprocessor),
      ("regressor", elastic_net)
  ])
  param_grid = {
        "regressor__alpha": [0.001, 0.01, 0.1, 1.0, 10.0],
        "regressor__l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]
    }
  tscv = TimeSeriesSplit(n_splits=5)
  random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=tscv, scoring='neg_root_mean_squared_error',n_iter=10, n_jobs=1)

  random_search.fit(X_train, y_train)
  best_params = random_search.best_params_
  print(f"Best paramters:{best_params}")
  best_model = random_search.best_estimator_
  y_pred_train = best_model.predict(X_train)
  rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
  mae_train = mean_absolute_error(y_train, y_pred_train)
  r2_train = r2_score(y_train, y_pred_train)
  y_pred = best_model.predict(X_val)
  rmse = np.sqrt(mean_squared_error(y_val, y_pred))
  mae = mean_absolute_error(y_val, y_pred)
  r2 = r2_score(y_val, y_pred)
  print(f'RMSE for training: {rmse_train}')
  print(f'R2 for training: {r2_train}')
  print(f'MAE for training: {mae_train}')

  print(f'RMSE: {rmse}')
  print(f'R2: {r2}')
  print(f'MAE: {mae}')
  return model


In [ ]:
model_en = elastic_net_model(training_df, validation_df, target_col='Oxygen')

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2277863.4694107994, tolerance: 1628.5119620611356
  model = cd_fast.sparse_enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5513257.371299884, tolerance: 3729.224071665968
  model = cd_fast.sparse_enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9764358.001576375, tolerance: 5666.532873166675
  model = cd_fast.sparse_enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:656: Conver

Best paramters:{'regressor__l1_ratio': 0.5, 'regressor__alpha': 0.001}
RMSE for training: 35.952875514070406
R2 for training: 0.6516904495060571
MAE for training: 21.009451734247023
RMSE: 42.399076367897216
R2: 0.5392769127359354
MAE: 26.185307814285363


Random forest

In [ ]:

from sklearn.ensemble import RandomForestRegressor
def random_forest_model(training_df,validation_df,target_col='Oxygen'):
  X_train=training_df.drop(target_col, axis=1)

  y_train=training_df[target_col]

  X_val=validation_df.drop(target_col, axis=1)

  y_val=validation_df[target_col]

  preprocessor = ColumnTransformer(
      transformers=[
          ("num", Pipeline([
              ("imputer", SimpleImputer(strategy="median")),
              ("scaler", StandardScaler())
          ]), selector(dtype_include=np.number)),
          ("cat", Pipeline([
              ("imputer", SimpleImputer(strategy="most_frequent")),
              ("ohe", OneHotEncoder(handle_unknown="ignore"))
          ]), selector(dtype_exclude=np.number)),
      ],
      remainder="drop"
  )



  rand_forest = RandomForestRegressor()
  model = Pipeline([
      ("preprocessor", preprocessor),
      ("regressor", rand_forest)
  ])

  param_grid = {
        'regressor__n_estimators': [100, 200, 300],
        'regressor__max_depth': [None, 10, 20],
        'regressor__min_samples_split': [2, 5, 10],
        'regressor__min_samples_leaf': [1, 2, 4],
        'regressor__bootstrap': [True, False]
    }
  tscv = TimeSeriesSplit(n_splits=5)
  random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=tscv, scoring='neg_root_mean_squared_error',n_iter=10, n_jobs=1)

  random_search.fit(X_train, y_train)
  best_params = random_search.best_params_
  print(f"Best paramters:{best_params}")
  best_model = random_search.best_estimator_
  y_pred_train = best_model.predict(X_train)
  rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
  mae_train = mean_absolute_error(y_train, y_pred_train)
  r2_train = r2_score(y_train, y_pred_train)
  y_pred = best_model.predict(X_val)
  rmse = np.sqrt(mean_squared_error(y_val, y_pred))
  mae = mean_absolute_error(y_val, y_pred)
  r2 = r2_score(y_val, y_pred)
  print(f'RMSE for training: {rmse_train}')
  print(f'R2 for training: {r2_train}')
  print(f'MAE for training: {mae_train}')

  print(f'RMSE: {rmse}')
  print(f'R2: {r2}')
  print(f'MAE: {mae}')
  return model


In [ ]:
model_rf = random_forest_model(training_df, validation_df, target_col='Oxygen')

LGBM

In [ ]:
!pip install lightgbm
from lightgbm import LGBMRegressor
def lgbm_model(training_df,validation_df,target_col='Oxygen'):
  X_train=training_df.drop(target_col, axis=1)

  y_train=training_df[target_col]

  X_val=validation_df.drop(target_col, axis=1)

  y_val=validation_df[target_col]

  preprocessor = ColumnTransformer(
      transformers=[
          ("num", Pipeline([
              ("imputer", SimpleImputer(strategy="median")),
              ("scaler", StandardScaler())
          ]), selector(dtype_include=np.number)),
          ("cat", Pipeline([
              ("imputer", SimpleImputer(strategy="most_frequent")),
              ("ohe", OneHotEncoder(handle_unknown="ignore"))
          ]), selector(dtype_exclude=np.number)),
      ],
      remainder="drop"
  )



  lgbm = LGBMRegressor()
  model = Pipeline([
      ("preprocessor", preprocessor),
      ("regressor", lgbm)
  ])

  param_grid = {
      'regressor__num_leaves': [20, 31, 40],
      'regressor__max_depth': [5, 7, -1],
      'regressor__learning_rate': [0.01, 0.05, 0.1],
      'regressor__n_estimators': [100, 200, 500],
      'regressor__colsample_bytree': [0.7, 0.8, 0.9],
      'regressor__subsample': [0.7, 0.8, 0.9],
  }
  tscv = TimeSeriesSplit(n_splits=5)
  random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=tscv, scoring='neg_root_mean_squared_error',n_iter=10, n_jobs=1)

  random_search.fit(X_train, y_train)
  best_params = random_search.best_params_
  print(f"Best paramters:{best_params}")
  best_model = random_search.best_estimator_
  y_pred_train = best_model.predict(X_train)
  rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
  mae_train = mean_absolute_error(y_train, y_pred_train)
  r2_train = r2_score(y_train, y_pred_train)
  y_pred = best_model.predict(X_val)
  rmse = np.sqrt(mean_squared_error(y_val, y_pred))
  mae = mean_absolute_error(y_val, y_pred)
  r2 = r2_score(y_val, y_pred)
  print(f'RMSE for training: {rmse_train}')
  print(f'R2 for training: {r2_train}')
  print(f'MAE for training: {mae_train}')

  print(f'RMSE: {rmse}')
  print(f'R2: {r2}')
  print(f'MAE: {mae}')
  return model


In [ ]:
model_lgbm = lgbm_model(training_df, validation_df, target_col='Oxygen')

Neural Network

In [ ]:
from sklearn.neural_network import MLPRegressor
def nn_model(training_df,validation_df,target_col='Oxygen'):
  X_train=training_df.drop(target_col, axis=1)

  y_train=training_df[target_col]

  X_val=validation_df.drop(target_col, axis=1)

  y_val=validation_df[target_col]

  preprocessor = ColumnTransformer(
      transformers=[
          ("num", Pipeline([
              ("imputer", SimpleImputer(strategy="median")),
              ("scaler", StandardScaler())
          ]), selector(dtype_include=np.number)),
          ("cat", Pipeline([
              ("imputer", SimpleImputer(strategy="most_frequent")),
              ("ohe", OneHotEncoder(handle_unknown="ignore"))
          ]), selector(dtype_exclude=np.number)),
      ],
      remainder="drop"
  )



  mlp = MLPRegressor(
        hidden_layer_sizes=(256, 128,64),
        activation='relu',
        solver='adam',
        max_iter=500,
        random_state=42
    )
  model = Pipeline([
      ("preprocessor", preprocessor),
      ("regressor", mlp)
  ])

  param_grid = {
      'regressor__hidden_layer_sizes': [(50,), (100, 50), (200, 100, 50)],
      'regressor__activation': ['relu', 'tanh', 'logistic'],
      'regressor__solver': ['adam', 'sgd'],
      'regressor__alpha': [0.0001, 0.001, 0.01],
      'regressor__learning_rate_init': [0.001, 0.01],
      'regressor__max_iter': [200, 500]
  }

  tscv = TimeSeriesSplit(n_splits=5)
  random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=tscv, scoring='neg_root_mean_squared_error',n_iter=10, n_jobs=1)

  random_search.fit(X_train, y_train)
  best_params = random_search.best_params_
  print(f"Best paramters:{best_params}")
  best_model = random_search.best_estimator_
  y_pred_train = best_model.predict(X_train)
  rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
  mae_train = mean_absolute_error(y_train, y_pred_train)
  r2_train = r2_score(y_train, y_pred_train)
  y_pred = best_model.predict(X_val)
  rmse = np.sqrt(mean_squared_error(y_val, y_pred))
  mae = mean_absolute_error(y_val, y_pred)
  r2 = r2_score(y_val, y_pred)
  print(f'RMSE for training: {rmse_train}')
  print(f'R2 for training: {r2_train}')
  print(f'MAE for training: {mae_train}')

  print(f'RMSE: {rmse}')
  print(f'R2: {r2}')
  print(f'MAE: {mae}')
  return model


In [ ]:
model_nn = nn_model(training_df, validation_df, target_col='Oxygen')

XGBoosting

In [ ]:
from xgboost import XGBRegressor

In [ ]:
from sklearn.neural_network import MLPRegressor
def xgb_model(training_df,validation_df,target_col='Oxygen'):
  X_train=training_df.drop(target_col, axis=1)

  y_train=training_df[target_col]

  X_val=validation_df.drop(target_col, axis=1)

  y_val=validation_df[target_col]

  preprocessor = ColumnTransformer(
      transformers=[
          ("num", Pipeline([
              ("imputer", SimpleImputer(strategy="median")),
              ("scaler", StandardScaler())
          ]), selector(dtype_include=np.number)),
          ("cat", Pipeline([
              ("imputer", SimpleImputer(strategy="most_frequent")),
              ("ohe", OneHotEncoder(handle_unknown="ignore"))
          ]), selector(dtype_exclude=np.number)),
      ],
      remainder="drop"
  )



  xgb = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        random_state=42,
        objective='reg:squarederror',
        n_jobs=-1
    )
  model = Pipeline([
      ("preprocessor", preprocessor),
      ("regressor", xgb)
  ])

  param_grid = {
      'regressor__n_estimators': [100, 200, 300],
      'regressor__learning_rate': [0.01, 0.1, 0.2],
      'regressor__max_depth': [3, 5, 7],
      'regressor__subsample': [0.7, 0.9],
      'regressor__colsample_bytree': [0.7, 0.9],
      'regressor__gamma': [0, 0.1, 0.2],
      'regressor__reg_alpha': [0, 0.005, 0.01],
      'regressor__reg_lambda': [1, 1.5, 2]
  }
  tscv = TimeSeriesSplit(n_splits=5)
  random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=tscv, scoring='neg_root_mean_squared_error',n_iter=10, n_jobs=1)

  random_search.fit(X_train, y_train)
  best_params = random_search.best_params_
  print(f"Best paramters:{best_params}")
  best_model = random_search.best_estimator_
  y_pred_train = best_model.predict(X_train)
  rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
  mae_train = mean_absolute_error(y_train, y_pred_train)
  r2_train = r2_score(y_train, y_pred_train)
  y_pred = best_model.predict(X_val)
  rmse = np.sqrt(mean_squared_error(y_val, y_pred))
  mae = mean_absolute_error(y_val, y_pred)
  r2 = r2_score(y_val, y_pred)
  print(f'RMSE for training: {rmse_train}')
  print(f'R2 for training: {r2_train}')
  print(f'MAE for training: {mae_train}')

  print(f'RMSE: {rmse}')
  print(f'R2: {r2}')
  print(f'MAE: {mae}')
  return model


In [ ]:
model_xgb = xgb_model(training_df, validation_df, target_col='Oxygen')

# Model Comparison

Linear Regression:
Random Forest:
LGBM:
Neural Network:
XG Boosting:

This best model was

# Feature Importance

#Ablation Study

# Insights


Based on our findings